In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Create a DataFrame with 30 rows and the specified attributes
data = {
    "name": [f"Recipe {i}" for i in range(1, 51)],
    "Meat": np.random.randint(100, 500, size=50),
    "Chicken": np.random.randint(100, 500, size=50),
    "Sugar": np.random.randint(10, 100, size=50),
    "Tomato": np.random.randint(50, 300, size=50),
    "Olive Oil": np.random.randint(10, 50, size=50)
}

recipe_map = pd.DataFrame(data)
recipe_map.head()

,name,Meat,Chicken,Sugar,Tomato,Olive Oil
0,Recipe 1,405,470,16,72,36
1,Recipe 2,309,387,38,119,27
2,Recipe 3,168,369,65,169,26
3,Recipe 4,199,233,25,206,39
4,Recipe 5,337,398,45,142,21


In [3]:
# Generate a separate DataFrame for calorie values
calories = np.random.randint(200, 800, size=50)  # Random calorie values for recipes
recipe_calorie = pd.DataFrame({
    "name": recipe_map["name"],
    "calories": calories
})

recipe_calorie.head()

,name,calories
0,Recipe 1,794
1,Recipe 2,448
2,Recipe 3,310
3,Recipe 4,295
4,Recipe 5,365


In [4]:
# Extract all columns except the first one (name) for similarity calculation
ingredients = recipe_map.iloc[:, 1:]

# Compute cosine similarity matrix
similarity = cosine_similarity(ingredients)

In [5]:
# Base recommendation function
def recommend_similar_recipes(name, filter_func=None, top_n=10):
    # Check if the recipe exists
    if name not in recipe_map['name'].values:
        return f"Recipe '{name}' not found."
    
    # Get the index of the recipe
    recipe_id = recipe_map[recipe_map['name'] == name].index[0]
    
    # Get similarity scores for the given recipe
    sim_scores = list(enumerate(similarity[recipe_id]))
    
    # Sort scores in descending order of similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get indices of top_n most similar recipes
    sim_scores = sim_scores[1:]  # Exclude the recipe itself
    
    # Apply calorie filter if provided
    if filter_func:
        sim_scores = [score for score in sim_scores if filter_func(score[0])]
    
    # Get the top_n indices after filtering
    sim_scores = sim_scores[:top_n]
    recipe_indices = [i[0] for i in sim_scores]
    
    # Return the names of the recommended recipes
    return recipe_map['name'].iloc[recipe_indices].tolist()

# Recommendation function for higher calorie recipes
def recommend_higher_calorie_recipe(name, top_n=10):
    # Get the calorie value of the target recipe
    target_calorie = recipe_calorie.loc[recipe_calorie['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(name, filter_func=lambda idx: recipe_calorie.loc[idx, 'calories'] > target_calorie, top_n=top_n)

# Recommendation function for lower calorie recipes
def recommend_lower_calorie_recipe(name, top_n=10):
    # Get the calorie value of the target recipe
    target_calorie = recipe_calorie.loc[recipe_calorie['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(name, filter_func=lambda idx: recipe_calorie.loc[idx, 'calories'] < target_calorie, top_n=top_n)


In [6]:
# Test the recommender systems
print("Similar recipes with higher calories:")
print(recommend_higher_calorie_recipe('Recipe 1', top_n=5))

print("\nSimilar recipes with lower calories:")
print(recommend_lower_calorie_recipe('Recipe 1', top_n=5))

Similar recipes with higher calories:
['Recipe 26']

Similar recipes with lower calories:
['Recipe 21', 'Recipe 28', 'Recipe 39', 'Recipe 29', 'Recipe 2']


In [8]:
# Locate the rows of the specific recipes by their names
recipes_to_check = ['Recipe 1', 'Recipe 26', 'Recipe 21', 'Recipe 28', 'Recipe 39', 'Recipe 2']
selected_recipes = recipe_map.loc[recipe_map['name'].isin(recipes_to_check)]
selected_calories = recipe_calorie.loc[recipe_calorie['name'].isin(recipes_to_check)]

# Display the selected rows
print("\nSelected recipes:")
print(selected_recipes)

print("\nSelected calories:")
print(selected_calories)


Selected recipes:
         name  Meat  Chicken  Sugar  Tomato  Olive Oil
0    Recipe 1   405      470     16      72         36
1    Recipe 2   309      387     38     119         27
20  Recipe 21   427      435     54      79         14
25  Recipe 26   415      251     36      61         27
27  Recipe 28   396      395     14     112         35
38  Recipe 39   344      393     24     119         15

Selected calories:
         name  calories
0    Recipe 1       794
1    Recipe 2       448
20  Recipe 21       649
25  Recipe 26       797
27  Recipe 28       693
38  Recipe 39       298
